In [1]:
import torchcodec
print(torchcodec.__version__)


0.9.1


In [2]:
from datasets import Audio
from datasets import load_dataset

ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", split="validation")

ds = ds.cast_column("audio", Audio(sampling_rate=16000))
sample = ds[0]["audio"]["array"]

print(sample.shape)


/home/bishwa/Unversity/Audio_LLM_Memory/audio_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating validation split: 100%|██████████| 73/73 [00:00<00:00, 1743.44 examples/s]

(93680,)


In [8]:
ds[0]['audio']['array']

array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
       0.0010376 ], shape=(93680,), dtype=float32)

In [3]:
sample

array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
       0.0010376 ], shape=(93680,), dtype=float32)